## モデルの機能を拡張する

LLMは非常に強力なツールですが、訓練データを通じて得られた知識や情報の範囲内でのみ機能します。しかし、訓練データに含まれていない質問をしなければならない場合はどうでしょう？または、訓練データには含まれていないが、それに関連する質問をしなければならない場合はどうでしょう？

この問題を解決する方法はいくつかあり、持っているリソースやそれにかけられる時間やお金によって異なります。以下にその例を示します：

- 必要な情報を含めるためにモデルを完全に再学習する。LLMに関しては、数百から数千のGPUを何週間も稼働させることができる限られた企業にしか不可能です。
- 新しい情報でモデルをファインチューニングする。ファインチューニングの場合必要なリソースは大きく減り、数日または数時間で行うことができます（モデルのサイズに依存）。ただし、モデルを完全に再学習しないため、新しい情報が回答に完全に取り込まれない可能性があります。ファインチューニングは特定のコンテキストや語彙の理解を向上させるのに優れています。頻繁に情報の更新がある場合、その度にモデルをファインチューニングし、デプロイし直す必要があります。
- 新しい情報をデータベースに保存し、クエリに関連する文章を検索して、その結果をLLMに送信するコンテキストとして追加します。この技術は**Retrieval Augmented Generation（RAG）**と呼ばれています。この方法は、モデルを再学習したりファインチューニングしたりせずに、新しい知識を利用でき、いつでも簡単に更新できます。

私たちはすでに[Milvus](https://milvus.io/)を使用してベクトルデータベースを準備しており、[California Driver's Handbook](https://www.dmv.ca.gov/portal/handbook/california-driver-handbook/)（カリフォルニアの運転者向けハンドブック）の内容を[エンべディング](https://www.ibm.com/topics/embedding)して保存しています。

このノートブックでは、RAGを使用して**請求文章に関するいくつかのクエリを行い**、追加された知識がどのように役立つかを確認します。

### 必要なライブラリとインポート

Labの指示に従って適切なワークベンチイメージを選択して起動した場合、必要なすべてのライブラリがすでにインストールされているはずです。もしインストールされていない場合は、次のセルの最初の行のコメントを外して正しいパッケージをすべてインストールしてください。その後、必要なライブラリをインポートします。

In [ ]:
# !pip install --no-cache-dir --no-dependencies --disable-pip-version-check -r requirements.txt # 正しいワークベンチイメージを選択していない場合のみコメントを外してください

import json
import os
import httpx
from os import listdir
from os.path import isfile, join
from langchain.chains import LLMChain
from langchain.chains import RetrievalQA, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.llms import VLLMOpenAI
from langchain_community.vectorstores import Milvus
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts.chat import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate
)
from milvus_retriever_with_score_threshold import MilvusRetrieverWithScoreThreshold

# 埋め込みモデルをダウンロードする際の警告を無効にする
import transformers
transformers.logging.set_verbosity_error()

### Langchainの設定

Langchainを使用してタスクパイプラインを定義します。

まず、クエリを送信する**LLM**を定義しましょう。

In [ ]:
# LLM推論APIのURL
inference_server_url = "_INFERENCE_URL_LLM_"

# LLMの定義
llm = VLLMOpenAI(
    openai_api_key="EMPTY",
    openai_api_base= f"{inference_server_url}/v1",
    model_name="elyza",
    top_p=0.92,
    temperature=0.01,
    max_tokens=512,
    presence_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    async_client=httpx.AsyncClient(verify=False),
    http_client=httpx.Client(verify=False)
)

次に、California Driver Handbookを準備して保存している**ベクトルデータベース**への接続を定義します。

In [ ]:
# 最初に、ハンドブックを処理するために使用したエンべディングを定義します
model_kwargs = {"trust_remote_code": True}
embeddings = HuggingFaceEmbeddings(
            model_name="nomic-ai/nomic-embed-text-v1",
            model_kwargs=model_kwargs,
            show_progress=False,
        )

# 次に、Milvusベクトルデータベースから関連データを取得するためのリトリーバーを定義します
retriever = MilvusRetrieverWithScoreThreshold(
            embedding_function=embeddings,
            collection_name="california_driver_handbook_1_0",
            collection_description="",
            collection_properties=None,
            connection_args={
                "host": "vectordb-milvus.ic-shared-milvus.svc.cluster.local",
                "port": "19530",
                "user": "root",
                "password": "Milvus",
            },
            consistency_level="Session",
            search_params=None,
            k=4,
            score_threshold=0.99,
            metadata_field="metadata",
            text_field="page_content",
        )

次に、クエリを作成するために使用するテンプレートを定義します。この**テンプレート**には、**References**（参照）セクションが含まれていることに注意してください。ベクトルデータベースから返されたドキュメントが挿入されるのはこのセクションです。

In [ ]:
system_template_string = """
あなたは、親切で、礼儀正しく、正直なアシスタントです。
常に気配りと尊重をもって接し、真摯にサポートします。できる限り有用な返答を提供しますが、安全を確保します。
有害で、倫理に反する、偏見のある、または否定的な内容は避けます。返答が公正でポジティブなものであることを確認します。
"""

user_template_string = """
与えられた文章と参照情報の内容をもとに、与えられた質問に答えてください。

### 文章:
{text}

### 参照情報:
{context}

参照して得られた情報は日本語に変換して下さい。

### 質問:
{query}

回答は日本語で実施して下さい。

### 回答:
"""

system_template = SystemMessagePromptTemplate.from_template(system_template_string)
user_template = HumanMessagePromptTemplate.from_template(user_template_string)

PROMPT = ChatPromptTemplate.from_messages([system_template, user_template])

モデルに問い合わせる準備ができました！
claimsフォルダには、請求文章の例が記載されたJSONファイルがあります。最初の請求文章を読み込み、それに関連する質問をします。

In [ ]:
# 請求文章の読み取り

filename = 'claims/claim1.json'

# JSONファイルを開く
with open(filename, 'r') as file:
    data = json.load(file)
claim = data["content"]

## LLMの実行、追加知識なし
まずはこれまで通りベクトルデータベースの助けなしに、請求文章に関する最初のクエリを実行しましょう。

In [ ]:
# クエリを作成して送信します。

query = "ダニエルは赤信号で通過することを許可されていましたか？"

conversation = LLMChain(llm=llm,
                        prompt=PROMPT,
                        verbose=False
                        )
resp = conversation.predict(context="", query=query, text=claim)

モデルが一般常識として交通規制に関しての理解があることを確認できます。

### ## LLMの実行、追加知識あり

同じプロンプトとクエリを使用しますが、RAGによりモデルがCalifornia Driver Handbookにアクセスできるようになります。

In [ ]:
# クエリを作成して送信します。

query = "ダニエルは赤信号で通過することを許可されていましたか？"
search_query = "Was Daniel allowed to pass at the red light?"

question_answer_chain = create_stuff_documents_chain(llm, PROMPT)
chain = create_retrieval_chain(retriever, question_answer_chain)

resp = chain.invoke({"query": query,"input": search_query, "text": claim})

モデルは、**赤信号は「停止」を意味する**という情報源を直接参照することができました。

情報の取得元についてベクトルデータベースの回答に関連するソースを確認できます。

In [ ]:
def format_sources(input_list):
        sources = ""
        if len(input_list) != 0:
            sources += input_list[0].metadata["source"] + ', page: ' + str(input_list[0].metadata["page"])
            page_list = [input_list[0].metadata["page"]]
            for item in input_list:
                if item.metadata["page"] not in page_list: # 重複を避ける
                    page_list.append(item.metadata["page"])
                    sources += ', ' + str(item.metadata["page"])
        return sources

results = format_sources(resp['context'])

print(results)

これで完了です！私たちは、外部知識を使ってLLMを補完する方法を学びました！